<a href="https://colab.research.google.com/github/ivaibhavjindal/helper-material-cheatsheets/blob/vaibhav/Notebooks/Save_Load_and_Export_Keras_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save, Load and Export Models in Keras

## Task 1: Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os

print('This notebook works with TensorFlow version:', tf.__version__)

folders = ['tmp', 'models', 'model_name', 'weights']
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

print(os.listdir('.'))

This notebook works with TensorFlow version: 2.2.0
['.config', 'weights', 'model_name', 'tmp', 'models', 'sample_data']


## Task 2: Create Model

In [2]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 3: Data Preprocessing

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = np.reshape(x_train, (x_train.shape[0], 784))/255.
x_test = np.reshape(x_test, (x_test.shape[0], 784))/255.

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

4423680/4422102 [==============================] - 0s 0us/step


## Task 4: Model Checkpoint During Training

In [4]:
checkpoint_dir = 'weights/'

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(checkpoint_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}'),
            monitor='val_acc', save_weights_only=True, save_best_only=True
        )
    ]
)

Epoch 1/2
118/118 [==============================] - 1s 12ms/step - loss: 0.6759 - acc: 0.7724 - val_loss: 0.4878 - val_acc: 0.8302
Epoch 2/2
118/118 [==============================] - 1s 10ms/step - loss: 0.4322 - acc: 0.8494 - val_loss: 0.4412 - val_acc: 0.8499


In [5]:
os.listdir(checkpoint_dir)

['epoch_01_acc_0.8302.index',
 'checkpoint',
 'epoch_01_acc_0.8302.data-00000-of-00001',
 'epoch_02_acc_0.8499.index',
 'epoch_02_acc_0.8499.data-00000-of-00001']

## Task 5: Load Weights

In [6]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.387512683868408, 0.11620000004768372]


In [8]:
model.load_weights('weights/epoch_02_acc_0.8499')
print(model.evaluate(x_test, y_test, verbose=False))

[0.44117480516433716, 0.8499000072479248]


## Task 6: Saving Complete Model During Training

In [9]:
models_dir = 'models'

model = create_model()

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(models_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}.h5'),
            monitor='val_acc', save_weights_only=False, save_best_only=False
        )
    ]
)

Epoch 1/2
118/118 [==============================] - 1s 11ms/step - loss: 0.6705 - acc: 0.7751 - val_loss: 0.4872 - val_acc: 0.8270
Epoch 2/2
118/118 [==============================] - 1s 10ms/step - loss: 0.4286 - acc: 0.8505 - val_loss: 0.4404 - val_acc: 0.8445


In [10]:
os.listdir(models_dir)

['epoch_01_acc_0.8270.h5', 'epoch_02_acc_0.8445.h5']

## Task 7: Load Models

In [11]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.317486524581909, 0.1387999951839447]


In [12]:
model = tf.keras.models.load_model('models/epoch_02_acc_0.8445.h5')
model.summary()
print(model.evaluate(x_test, y_test, verbose=False))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
[0.4403533637523651, 0.8445000052452087]


## Task 8: Manually Saving Weights and Models

In [13]:
model.save_weights('tmp/manually_saved')
print(os.listdir('tmp'))

['checkpoint', 'manually_saved.data-00000-of-00001', 'manually_saved.index']


In [14]:
model.save('tmp/manually_saved_model.h5')
print(os.listdir('tmp'))

['checkpoint', 'manually_saved.data-00000-of-00001', 'manually_saved_model.h5', 'manually_saved.index']


## Task 9: Exporting and Restoring SavedModel Format

In [15]:
model.save('model_name')
print(os.listdir('model_name'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_name/assets
['assets', 'saved_model.pb', 'variables']


In [16]:
model = tf.keras.models.load_model('model_name')
print(model.evaluate(x_test, y_test, verbose=False))

[0.4403533637523651, 0.8445000052452087]
